<a href="https://colab.research.google.com/github/LeandroJorquera/synthetic_L1/blob/main/ORF2p_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Extracting information from Dfam ORF2 dataset (first test)

In [ ]:
import pandas as pd

# -- read dataset and exploring keys
df=pd.read_csv("/content/LINE_v2.csv", engine='python', on_bad_lines='skip')
#print(df)

# printing first row
#print(df.iloc[0])

# splitting df into keys
parameters = df.keys() 
#print(parameters)

# -- obtaining consensus sequence and CN (gathering all and gathering non-redundant) -- #
# extract dict that contains info (it is not a dict but a STRING)
seq_dict = parameters[19]
cn_dict = parameters[17]
prot_dict = parameters[21]
#print(type(seq_dict))
#print(type(cn_dict))

# convert str into dicts for extracting seq and CN
Dict_seq = eval(seq_dict)
Dict_cn = eval(cn_dict)
Dict_prot = eval(prot_dict)
#print(Dict_seq)
#print(Dict_cn)
#print(Dict_prot)
for key, value in Dict_seq.items():
  #print(key,value)
  if key == 'name':
    print(value)
  if key == 'consensus_sequence':
    print(value)
for key, value in Dict_cn.items():
     if key == 'hmm': #it is a dict of key 'hmm' and the rest is the value
       dict_in_dict = value
       print(dict_in_dict['gathering_nonredundant'])
       print (dict_in_dict['gathering_all']) 

# -- iterating over the dataset



# Reading and sorting dataset

In [ ]:
# -- Read dataset skipping empty lines
import pandas as pd
df=pd.read_csv("/content/LINE_v2.csv", engine='python', on_bad_lines='skip')  # 
#df.head(5)
n = sum(1 for line in open ('/content/LINE_v2.csv'))-1    #calculate number of rows
print(n)

9700


In [ ]:
# -- Select desired columns and rename

sorted_df = df.iloc[:,[18,2,7,17]]   # 2:Dfam ID // 7:DNA_seq // 17: counts info // 18:ID + Dfam entry
sorted_df.columns = ['ID','Length','DNA_seq','Counts']
sorted_df.head(5)

# -- Select columns matching required criteria (length)
filtered_df = sorted_df.loc[sorted_df['Length']>=3000]
# -- removing duplicates
filtered_df.drop_duplicates(subset=['ID'], keep=False)
#len(filtered_df)  


# Retrieving count number

In [ ]:
# select desired columns and create empty output dict
name = filtered_df['ID']  
counts = filtered_df['Counts']
output_dict = {}

# iterating over dataset
for i in range(len(filtered_df)):
    ID = name.iloc[i]
    if type(counts.iloc[i]) == str: # must be string to be converted to dict
      hmm_dict = eval(counts.iloc[i])
      count_list = list(hmm_dict.values())  
      count_dict = eval(str(count_list[0])) 
      all = count_dict['gathering_all']
      non_redun = count_dict['gathering_nonredundant']
      #print(ID , all, non_redun)
      output_dict[ID] = (all,non_redun) # associate both number to each ID

# Display output dict
#output_dict

# Make a df and ID column is set as index
count_df = pd.DataFrame.from_dict(output_dict, orient='index') 
#count_df

# Resetting index to displace ID col
count_df2 = count_df.reset_index()
#count_df2

# Renaming columns
count_df2.columns = ['ID','All_counts', 'Non-redundant_counts']
count_df2

,ID,All_counts,Non-redundant_counts
0,L1M1_5endDF0000227,96690,11185
1,L1M1_orf2DF0000228,277575,11129
2,L1M2_orf2DF0000236,296693,17500
3,L1M3_orf2DF0000244,299220,21747
4,L1M4b_5endDF0000248,134213,4672
...,...,...,...
781,MacSob-1.188DF0277952,188329,35322
782,ManJav-4.143DF0277991,347866,141913
783,ManJav-6.20DF0278000,397516,179141
784,ManLeu-1.25DF0278009,387912,32927


# Protein extraction with simpler ORFinder

In [ ]:
# -- Load modules
#!pip install Bio
from Bio import SeqIO
from Bio.Seq import Seq
# -- Extracting all protein sequences from DNA with simpler finder
prot_list = []
prot_dict = {}
name = filtered_df['ID']  
DNA = filtered_df['DNA_seq'] 
for i in range(len(filtered_df)):
  ID = name.iloc[i]
  if type(DNA.iloc[i]) == str:
    record = Seq(DNA.iloc[i])
  for strand, nuc in [(+1, record), (-1, record.reverse_complement())]:   # explore seq and rev complement
       for frame in range(3): #explore all 3 ORFs
         min_pro_len = 1000
         table = 1
         length = 3 * ((len(record) - frame) // 3)  # Multiple of three
         for pro in nuc[frame : frame + length].translate(table).split("*"):
             if len(pro) >= min_pro_len:
               #print( "%s - %s...%s - length %i," % (ID, pro[:20], pro[-3:], len(pro)))
               prot_list.append(str(pro))
               prot_dict[ID] = str(pro)

# Display output prot dict
#prot_dict

# Make a df and ID column is set as index
prot_df = pd.DataFrame.from_dict(prot_dict, orient='index') 
# Resetting index to displace ID col
final_df = prot_df.reset_index()

# Using drop() function to delete last row and renaming cols
final_df.columns = ['ID','Seq']
#final_df.drop(index=final_df.index[-1], inplace=True)
final_df


,ID,Seq
0,L1M1_orf2DF0000228,MAGVSPYLSIITLNVNGLNSPIKRHRVAEWMKKQDPXICCLQETHF...
1,L1M2_orf2DF0000236,MAXVSPYLSIITLNVNGLNSPIKRHRXAEWIKKQDPTICCLQETHF...
2,L1M3_orf2DF0000244,MAIVSPXLSIITLNVNGLNSPIKRHRVAEWIKKQDPTICCLQETHF...
3,L1M4_orf2DF0000250,MADVNPTXSVITLNVNGLNTPIKRQRLAEWIKKHDPTICCLQETHF...
4,L1MD_orf2DF0000292,MADLSPNISIITLNVNGLNTPIKRQRLAEWIKKHDPTICCLQETHF...
...,...,...
414,LycPic-3.16DF0277930,IGIMMTLNSYLSIVTLNVNGLNDPIKRRRVSDWIKKQDPSICCLQE...
415,LycPic-4.88DF0277932,IGIMMTLNSYLSIVTLNVNGLNDPIKRRRVSDWIKKQDPSICCLQE...
416,ManJav-4.143DF0277991,KPKDMHRLKVKGWKKIFHANNXEKKAGVAVLVSDKIDFKTKKVTRD...
417,ManLeu-1.25DF0278009,PANIIMTGSNSHITILTLNVNGLNAPIKRHRLANWIKSQDPSVCCI...


In [ ]:
from google.colab import files

definitive_df = pd.merge(final_df,count_df2)
definitive_df
#definitive_df.to_csv('ORF2_dfam.csv')
#files.download('ORF2_dfam.csv')

,ID,Seq,All_counts,Non-redundant_counts
0,L1M1_orf2DF0000228,MAGVSPYLSIITLNVNGLNSPIKRHRVAEWMKKQDPXICCLQETHF...,277575,11129
1,L1M2_orf2DF0000236,MAXVSPYLSIITLNVNGLNSPIKRHRXAEWIKKQDPTICCLQETHF...,296693,17500
2,L1M3_orf2DF0000244,MAIVSPXLSIITLNVNGLNSPIKRHRVAEWIKKQDPTICCLQETHF...,299220,21747
3,L1M4_orf2DF0000250,MADVNPTXSVITLNVNGLNTPIKRQRLAEWIKKHDPTICCLQETHF...,313178,32089
4,L1MD_orf2DF0000292,MADLSPNISIITLNVNGLNTPIKRQRLAEWIKKHDPTICCLQETHF...,309154,18798
...,...,...,...,...
413,LycPic-1.114DF0277912,IGITMTLNSYLSIVTLNVNGLNDPIKRRRVSDWIKKQDPSICCLQE...,311969,13698
414,LycPic-3.16DF0277930,IGIMMTLNSYLSIVTLNVNGLNDPIKRRRVSDWIKKQDPSICCLQE...,378165,46655
415,LycPic-4.88DF0277932,IGIMMTLNSYLSIVTLNVNGLNDPIKRRRVSDWIKKQDPSICCLQE...,383825,91364
416,ManJav-4.143DF0277991,KPKDMHRLKVKGWKKIFHANNXEKKAGVAVLVSDKIDFKTKKVTRD...,347866,141913


# Exporting as fasta file from prot_dict

In [ ]:
# -- export dataset to fasta from prot_dict
from Bio import SeqIO

# output file name
output_file = 'ORF2list.fasta'

# open output file for writing
with open(output_file, 'w') as out_file:
  for seq_id, seq in prot_dict.items(): # loop through dictionary items 
    seq_record = SeqIO.SeqRecord(Seq(seq), id= str(seq_id)) # make sure to save ID as string
    SeqIO.write(seq_record, out_file, 'fasta')



# Protein extraction with ORFinder

In [ ]:
from pandas.core.groupby.groupby import DataFrame
# -- Load modules
#!pip install Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import BiopythonWarning

#record = SeqIO.read("NC_005816.fna", "fasta") #parse sequence
table = 1  #set table for aa
min_pro_len = 1000 #lower limit for protein length

def find_orfs_with_trans(seq, trans_table,min_protein_lenght):
  answer=[] #list of results
  seq_len = len(seq)
  for strand, nuc in ([+1, seq], (-1, seq.reverse_complement())):
    for frame in range(3):
      trans = nuc[frame:].translate(trans_table)
      trans_len = len(trans)
      aa_start = 0
      aa_end = 0
      while aa_start < trans_len:
        aa_end = trans.find("*",aa_start)
        if aa_end == -1:
          aa_end = trans_len
        if aa_end - aa_start >= min_protein_lenght:
          if aa_end == 1:
            start = frame + aa_start * 3
            end = min(seq_len, frame + aa_end * 3 +3)
          else:
            start = seq_len - frame - aa_end * 3 -3
            end = seq_len - frame - aa_start * 3
          answer.append ((start, end, strand, trans[aa_start:aa_end]))
        aa_start = aa_end + 1 

  answer.sort()  #sort the table for start base position
  return answer

# -- iterate over all elements in DNA_seq col
record_list=[]
for i in filtered_df['DNA_seq']:
  if type(i) == str:
    record = Seq(i)
    record_list.append(record)
    #print (record)
    orf_list = find_orfs_with_trans(record, table, min_pro_len) # -- call ORFinder
  else:
    continue
#len(record_list)
#len(orf_list)
for start, end, strand, pro in orf_list:  # -- show results 
  print ("%s ...%s - lenght %i, strand %i, %i:%i" % (pro[:20], pro[-3:], len(pro), strand, start, end))


In [ ]:
# -- iterate over all elements in DNA_seq col using complex ORFinder
record_list=[]
for i in filtered_df['DNA_seq']:
  if type(i) == str:
    record = Seq(i)
    record_list.append(record)
    #print (record)
    orf_list = find_orfs_with_trans(record, table, min_pro_len) # -- call ORFinder
  else:
    continue
#len(record_list)
#len(orf_list)
for start, end, strand, pro in orf_list:  # -- show results 
  print ("%s ...%s - lenght %i, strand %i, %i:%i" % (pro[:20], pro[-3:], len(pro), strand, start, end))

# Table supplementary ORF2/hits

In [ ]:
import pandas as pd
df_2 = pd.read_csv("/content/ORF2_suppl.csv", delimiter=';')
df_2.columns = ['species', 'ORF2_CN']
#print(df)
#df.head()
df_2.sort_values(by=['ORF2_CN'], ascending=False)


,species,ORF2_CN
0,Monodelphis domestica,1607
44,Balaenoptera\nacutorostrata scammoni,1338
84,Rhinopithecus roxellana,714
75,Mus musculus,629
350,Brassica napus,544
...,...,...
262,Schistosoma curassoni,0
263,Schistosoma haematobium,0
264,Schistosoma japonicum,0
265,Schistosoma mansoni,0


# Merge datasets

In [ ]:
# -- merging datasets according to species name
merged_df = pd.merge(df_2,sorted_df)
#len(merged_df)
#merged_df.head(10)

# -- Dropping columns not matching criteria (length)
valid_orf2 = merged_df.loc[(merged_df['length']>=3000) & (merged_df['length']<4000)]
valid_orf2
len(valid_orf2)

#merged_unique = merged_df.drop_duplicates(subset=['nickname'], keep=False)
#print(len(merged_unique))
#merged_unique


686